### Import Library

In [2]:
import pandas as pd
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler

### Extract Data

In [41]:
def extract_data():
    data = pd.read_csv('house_listings.csv')
    return data

if __name__ == '__main__':
    extracted_data = extract_data()
    extracted_data.to_excel('house_listings.xlsx')
    print(extracted_data)
    

           category    price currency     price_1m2  \
0      Köhnə tikili  137 000      AZN  2 630 AZN/m²   
1       Yeni tikili  183 000      AZN  1 790 AZN/m²   
2      Köhnə tikili  145 000      AZN  2 230 AZN/m²   
3      Köhnə tikili  190 000      AZN  2 000 AZN/m²   
4       Yeni tikili  294 000      AZN  1 550 AZN/m²   
...             ...      ...      ...           ...   
35498  Köhnə tikili  135 000      AZN  2 080 AZN/m²   
35499   Yeni tikili  245 000      AZN  2 720 AZN/m²   
35500   Yeni tikili  309 000      AZN  2 310 AZN/m²   
35501   Yeni tikili  292 000      AZN  2 150 AZN/m²   
35502  Köhnə tikili  130 000      AZN  2 170 AZN/m²   

                                                   title  \
0      Satılır 3 otaqlı köhnə tikili 52 m², 8-ci kilo...   
1      Satılır 2 otaqlı yeni tikili 102 m², Neftçilər m.   
2      Satılır 2 otaqlı köhnə tikili 65 m², Nərimanov r.   
3        Satılır 3 otaqlı köhnə tikili 95 m², Gənclik m.   
4         Satılır 3 otaqlı yeni tikili 

### Transform Data

In [22]:
def transform_data(data):
    #bersihkan data
    data['price'] = data['price'].astype(str)
    data['price'] = data['price'].str.replace(',','.')
    data['price'] = data['price'].str.replace(' ', '').astype(float)

    #normalisasi
    min_price = data['price'].min()
    max_price = data['price'].max()
    data['normalized_price'] = (data['price'] - min_price) / (max_price - min_price)

    #bersihkan data
    data['price_1m2'] = data['price_1m2'].astype(str)
    data['price_1m2'] = data['price_1m2'].str.replace(',','.')
    data['price_1m2'] = data['price_1m2'].str.split(' ', expand=True)[0].astype(float)
    
    #normalisasi
    min_price_1m2 = data['price_1m2'].min()
    max_price_1m2 = data['price_1m2'].max()
    data['normalized_price_1m2'] = (data['price_1m2'] - min_price_1m2) / (max_price_1m2 - min_price_1m2)
    
    #OHE
    ce_OHE = ce.OneHotEncoder(cols=['category', 'title_deed', 'repair', 'mortgage'])
    data = ce_OHE.fit_transform(data)

    data['is_missing_price'] = data['price'].apply(lambda x: 1 if x == '?' else 0)
    data = data[data['is_missing_price']==0]

    return data  

#exection
if __name__ == '__main__':
    extracted_data = pd.read_csv('house_listings.csv')
    transformed_data = transform_data(extracted_data)
    transformed_data.to_excel('transformed_house_listings.xlsx')
    print(transformed_data)

       category_1  category_2  category_3     price currency  price_1m2  \
0               1           0           0  137000.0      AZN        2.0   
1               0           1           0  183000.0      AZN        1.0   
2               1           0           0  145000.0      AZN        2.0   
3               1           0           0  190000.0      AZN        2.0   
4               0           1           0  294000.0      AZN        1.0   
...           ...         ...         ...       ...      ...        ...   
35498           1           0           0  135000.0      AZN        2.0   
35499           0           1           0  245000.0      AZN        2.0   
35500           0           1           0  309000.0      AZN        2.0   
35501           0           1           0  292000.0      AZN        2.0   
35502           1           0           0  130000.0      AZN        2.0   

                                                   title  \
0      Satılır 3 otaqlı köhnə tikili 52

In [23]:
#aggregasi data
def transform_data(data):
    #bersihkan data
    data['price'] = data['price'].astype(str)
    data['price'] = data['price'].str.replace(',','.')
    data['price'] = data['price'].str.replace(' ', '').astype(float)

    #normalisasi
    min_price = data['price'].min()
    max_price = data['price'].max()
    data['normalized_price'] = (data['price'] - min_price) / (max_price - min_price)

    #bersihkan data
    data['price_1m2'] = data['price_1m2'].astype(str)
    data['price_1m2'] = data['price_1m2'].str.replace(',','.')
    data['price_1m2'] = data['price_1m2'].str.split(' ', expand=True)[0].astype(float)
    
    #normalisasi
    min_price_1m2 = data['price_1m2'].min()
    max_price_1m2 = data['price_1m2'].max()
    data['normalized_price_1m2'] = (data['price_1m2'] - min_price_1m2) / (max_price_1m2 - min_price_1m2)
    
    return data

if __name__ == '__main__':
    extracted_data = pd.read_csv('house_listings.csv')
    transformed_data = transform_data(extracted_data)

    # Aggregasi Data
    aggregated_data = transformed_data.groupby(['room_number', 'category']).agg({'price': ['mean', 'median', lambda x: x.mode().iloc[0]]}).reset_index()
    aggregated_data.columns = ['room_number', 'category', 'mean_price', 'median_price', 'mode_price']

    # Simpan ke file Excel
    aggregated_data.to_excel('aggregated_house_listings.xlsx', index=False)

    print(aggregated_data)


    room_number      category    mean_price  median_price  mode_price
0           1.0  Köhnə tikili  9.134851e+04       88000.0     95000.0
1           1.0   Yeni tikili  1.222027e+05      120000.0    130000.0
2           2.0  Köhnə tikili  1.200298e+05      115000.0    120000.0
3           2.0   Yeni tikili  1.641581e+05      155000.0    145000.0
4           3.0  Köhnə tikili  1.666889e+05      153500.0    175000.0
5           3.0   Yeni tikili  2.553884e+05      245000.0    250000.0
6           4.0  Köhnə tikili  2.116755e+05      180000.0    165000.0
7           4.0   Yeni tikili  4.055463e+05      370000.0    350000.0
8           5.0  Köhnə tikili  2.765835e+05      201500.0    220000.0
9           5.0   Yeni tikili  6.278174e+05      520000.0    450000.0
10          6.0  Köhnə tikili  6.251208e+05      500000.0    275000.0
11          6.0   Yeni tikili  8.421765e+05      750000.0   1000000.0
12          7.0  Köhnə tikili  7.716250e+05      625000.0    165000.0
13          7.0   Ye